In [9]:
import os 
workdir="/Users/tr/Documents/GitHub/Learning-Agents"
os.chdir(workdir)

In [10]:
from PIL import Image
from pix2tex.cli import LatexOCR

img = Image.open('pdf_sparse/test_notes_page_1.png')
model = LatexOCR()
print(model(img))

/opt/anaconda3/envs/python39/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/opt/anaconda3/envs/python39/lib/python3.9/site-packages/albumentations/__init__.py:24: UserWarning: A new version of Albumentations is available: 2.0.8 (you have 1.4.24). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()
/opt/anaconda3/envs/python39/lib/python3.9/site-packages/pydantic/_internal/_serializers.py:44: UserWarning: Pydantic serializer warnings:
  PydanticSerializationUnexpectedValue(Expected `dict[str, any]` - serialized value may not be as expected [input_value=UniformParams(noise_type=... 0.058823529411764705)]), input_type=UniformParams])
  v = handler(item, index)
/opt/anaconda3/envs/python39/lib/py

download weights v0.0.1 to path /opt/anaconda3/envs/python39/lib/python3.9/site-packages/pix2tex/model/checkpoints


weights.pth: 100%|██████████| 97.4M/97.4M [00:19<00:00, 5.29Mb/s]
image_resizer.pth: 100%|██████████| 18.5M/18.5M [00:10<00:00, 1.88Mb/s]
[W NNPACK.cpp:64] Could not initialize NNPACK! Reason: Unsupported hardware.


\begin{array}{c}{{\mid_{\scriptstyle p_{*}}^{\quad n}}}\\ {{\bar{z}}}\\ {{\vdots}}\\ {{\bar{z}}}\\ {{\bar{z}}}\end{array}


In [11]:
model(img)

'\\begin{array}{c}{{\\mid_{\\scriptstyle p_{*}}^{\\scriptstyle p}}}\\\\ {{\\bar{z}}}\\\\ {{\\vdots}}\\\\ {{\\bar{z}}}\\\\ {{\\bar{z}}}\\end{array}'

In [21]:
import os
import sys
import re
import shutil
from PIL import Image
from paddleocr import PaddleOCR
from pix2tex.cli import LatexOCR

# 初始化 OCR 模型
ocr = PaddleOCR(use_angle_cls=True)
latex_ocr = LatexOCR()

# 简单的公式判断规则（示例）
def is_formula(text):
    # 这里用简单规则判别公式，可根据实际情况改进
    formula_patterns = [r'\=', r'\\frac', r'\(', r'\)', r'\+', r'\-', r'\*', r'\^', r'\[', r'\]', r'\\']
    for pattern in formula_patterns:
        if re.search(pattern, text):
            return True
    return False

# 主流程
image_path="pdf_sparse/test_notes_page_1.png"
output_dir="ocr_output"
if not os.path.exists(output_dir):
    os.makedirs(output_dir)


/var/folders/gk/zqytr9t577n5vwk871xhnpwh0000gn/T/ipykernel_60104/2669446842.py:10: DeprecationWarning: The parameter `use_angle_cls` has been deprecated and will be removed in the future. Please use `use_textline_orientation` instead.
  ocr = PaddleOCR(use_angle_cls=True)
Creating model: ('PP-LCNet_x1_0_doc_ori', None)
Using official model (PP-LCNet_x1_0_doc_ori), the model files will be automatically downloaded and saved in /Users/tr/.paddlex/official_models.
Creating model: ('UVDoc', None)
Using official model (UVDoc), the model files will be automatically downloaded and saved in /Users/tr/.paddlex/official_models.
Creating model: ('PP-LCNet_x0_25_textline_ori', None)
Using official model (PP-LCNet_x0_25_textline_ori), the model files will be automatically downloaded and saved in /Users/tr/.paddlex/official_models.
Creating model: ('PP-OCRv5_mobile_det', None)
Using official model (PP-OCRv5_mobile_det), the model files will be automatically downloaded and saved in /Users/tr/.paddlex/

In [22]:
# Step 1. 使用 PaddleOCR 检测文字区域
results = ocr.ocr(image_path)
image = Image.open(image_path)


/var/folders/gk/zqytr9t577n5vwk871xhnpwh0000gn/T/ipykernel_60104/2516359575.py:2: DeprecationWarning: Please use `predict` instead.
  results = ocr.ocr(image_path)


In [27]:
results[0]

{'input_path': 'pdf_sparse/test_notes_page_1.png',
 'page_index': None,
 'doc_preprocessor_res': {'input_path': None,
  'page_index': None,
  'input_img': array([[[255, ..., 255],
          ...,
          [255, ..., 255]],
  
         ...,
  
         [[255, ..., 255],
          ...,
          [255, ..., 255]]], dtype=uint8),
  'model_settings': {'use_doc_orientation_classify': True,
   'use_doc_unwarping': True},
  'angle': 0,
  'rot_img': array([[[255, ..., 255],
          ...,
          [255, ..., 255]],
  
         ...,
  
         [[255, ..., 255],
          ...,
          [255, ..., 255]]], dtype=uint8),
  'output_img': array([[[255, ..., 255],
          ...,
          [255, ..., 255]],
  
         ...,
  
         [[255, ..., 255],
          ...,
          [255, ..., 255]]], dtype=uint8)},
 'dt_polys': [array([[634,  58],
         ...,
         [631, 160]], dtype=int16),
  array([[140, 232],
         ...,
         [138, 328]], dtype=int16),
  array([[1217,  356],
         ...,
 

In [32]:
text_parts = []
formula_parts = []

index = 0
rec_texts = results[0].get("rec_texts", [])
rec_scores = results[0].get("rec_scores", [])
rec_polys = results[0].get("rec_polys", [])

# 遍历 OCR 结果
for text, score, box in zip(rec_texts, rec_scores, rec_polys):
    if is_formula(text):
        x_min = min(point[0] for point in box)
        y_min = min(point[1] for point in box)
        x_max = max(point[0] for point in box)
        y_max = max(point[1] for point in box)

        cropped = image.crop((x_min, y_min, x_max, y_max))
        formula_image_path = os.path.join(output_dir, f'formula_{index}.png')
        cropped.save(formula_image_path)
        formula_parts.append(formula_image_path)
        index += 1
    else:
        text_parts.append(text.strip())

# Step 2. 文字部分直接用 PaddleOCR 输出
text_content = '\n'.join(text_parts)

# Step 3. 公式部分用 latexocr 识别
latex_results = []
for formula_img in formula_parts:
    try:
        latex = latex_ocr(formula_img)
        latex_results.append(latex)
    except Exception as e:
        print(f"[错误] 处理 {formula_img} 时出错：{e}")
        latex_results.append("[公式识别失败]")

# Step 4. 输出合并
output_file = os.path.join(output_dir, 'output.tex')
with open(output_file, 'w', encoding='utf-8') as f:
    f.write("以下是文字部分：\n\n")
    f.write(text_content)
    f.write("\n\n以下是公式部分（LaTeX）：\n\n")
    for latex in latex_results:
        f.write(f"${latex}$\n\n")

print(f"输出已保存到 {output_file}")

[错误] 处理 ocr_output/formula_0.png 时出错：'str' object has no attribute 'copy'
[错误] 处理 ocr_output/formula_1.png 时出错：'str' object has no attribute 'copy'
[错误] 处理 ocr_output/formula_2.png 时出错：'str' object has no attribute 'copy'
[错误] 处理 ocr_output/formula_3.png 时出错：'str' object has no attribute 'copy'
[错误] 处理 ocr_output/formula_4.png 时出错：'str' object has no attribute 'copy'
[错误] 处理 ocr_output/formula_5.png 时出错：'str' object has no attribute 'copy'
[错误] 处理 ocr_output/formula_6.png 时出错：'str' object has no attribute 'copy'
[错误] 处理 ocr_output/formula_7.png 时出错：'str' object has no attribute 'copy'
[错误] 处理 ocr_output/formula_8.png 时出错：'str' object has no attribute 'copy'
[错误] 处理 ocr_output/formula_9.png 时出错：'str' object has no attribute 'copy'
[错误] 处理 ocr_output/formula_10.png 时出错：'str' object has no attribute 'copy'
[错误] 处理 ocr_output/formula_11.png 时出错：'str' object has no attribute 'copy'
[错误] 处理 ocr_output/formula_12.png 时出错：'str' object has no attribute 'copy'
[错误] 处理 ocr_output/formula_13.png 时

In [31]:
for line in results[0]:
    print(line)


input_path
page_index
doc_preprocessor_res
dt_polys
model_settings
text_det_params
text_type
text_rec_score_thresh
rec_texts
rec_scores
rec_polys
textline_orientation_angles
rec_boxes
